<a href="https://colab.research.google.com/github/sagihaider/EEG_Deep/blob/master/main_Single_Trial_Classification_Accuracy_all_subjects_FilteredData_Binary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/sagihaider/EEG_Deep.git

Cloning into 'EEG_Deep'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 79 (delta 11), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (79/79), done.
Checking out files: 100% (28/28), done.


In [2]:
import scipy.io as spio
import numpy as np
from importlib.machinery import SourceFileLoader

# EEGNet-specific imports
from EEG_Deep.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
K.set_image_data_format('channels_first')

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

# tools for plotting confusion matrices
from matplotlib import pyplot as plt
from scipy.signal import butter, lfilter

In [0]:
# Band-pass Filter
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = lfilter(b, a, data)
    return y


In [116]:
from numpy import zeros
K.clear_session()
cols = 2
rows = 9
acc_all = zeros([rows, cols])
X_tr_c12 = np.empty([288, 22, 1875])
X_ts_c12 = np.empty([288, 22, 1875])
result=[]

from itertools import combinations 
comb = combinations([1, 2, 3,4], 2) 
  # Print the obtained combinations 
bincomb=[]
for i in list(comb): 
    bincomb.append(i)

for x in range(1,2):
  fName = 'EEG_Deep/Data2A/Data_A0' + str(x) + 'T.mat'  # Load Data
  print(fName)
  mat = spio.loadmat(fName)
  r_X_tr = mat['cleanRawEEGData']
  r_y_tr = mat['cleanClassLabels']

  ### Filter Data ###
  for t in range(r_X_tr.shape[0]):
    tril = r_X_tr[t,:,:]
    # tril = tril.transpose()
    tril_filtered = butter_bandpass_filter(tril, 
                                              lowcut=4, 
                                              highcut=40, 
                                              fs=250,
                                              order=4)
    # tril_filtered = tril_filtered.transpose()
    X_tr_c12[t,:,:] = tril_filtered

  print("Filtering of Training Data Finished")
  ## Test Data Load 

  fName = 'EEG_Deep/Data2A/Data_A0' + str(x) + 'T.mat'  # Load Data
  print(fName)
  mat = spio.loadmat(fName)
  r_X_ts = mat['cleanRawEEGData']
  r_y_ts = mat['cleanClassLabels']
  for t in range(r_X_ts.shape[0]):
    tril = r_X_ts[t,:,:]
    # tril = tril.transpose()
    tril_filtered = butter_bandpass_filter(tril, 
                                              lowcut=4, 
                                              highcut=40, 
                                              fs=250,
                                              order=4)
    # tril_filtered = tril_filtered.transpose()
    X_ts_c12[t,:,:] = tril_filtered
  
  print("Filtering of Testing Data Finished")    

  for k, com in enumerate(bincomb):
      print(com)
      # # Find labels to specific class
      print("Finding labels in training data")
      class1indx = list(np.where(r_y_tr == com[0]))
      class2indx = list(np.where(r_y_tr == com[1]))
      c1=list(class1indx[0])
      c2=list(class2indx[0])
      y_tr_c12 = c1 + c2
      y_tr_c12.sort()
      # print(y_tr_c12)
      x_tr_12 = X_tr_c12[y_tr_c12,:,:]
      y_tr_12 = r_y_tr[y_tr_c12]
      # print(np.shape(x_tr_12))
      # print(np.shape(y_tr_12))
      # # Find labels to specific class
      print("Finding labels in testing data")
      class1indx = list(np.where(r_y_ts == com[0]))
      class2indx = list(np.where(r_y_ts == com[1]))
      c1=list(class1indx[0])
      c2=list(class2indx[0])
      y_ts_c12 = c1 + c2
      y_ts_c12.sort()
      # print(y_ts_c12)
      x_ts_12 = X_ts_c12[y_ts_c12,:,:]
      y_ts_12 = r_y_ts[y_ts_c12]
      # print(np.shape(x_ts_12))
      # print(np.shape(y_ts_12))
      del class1indx, class2indx, c1, c2
      # split data of each subject in training and validation
      X_train = x_tr_12[0:120,:,500:1250]
      Y_train = y_tr_12[0:120]
      X_val   = x_tr_12[120:,:,500:1250]
      Y_val   = y_tr_12[120:]
      # print(Y_val)

      print(np.shape(X_train))
      print(np.shape(Y_train))
      print(np.shape(X_val))
      print(np.shape(Y_val))
  
      # convert labels to one-hot encodings.
      Y_train      = np_utils.to_categorical(Y_train-1)
      Y_val       = np_utils.to_categorical(Y_val-1)
      print(Y_val)

      # kernels, chans, samples = 1, 22, 750
      # # convert data to NCHW (trials, kernels, channels, samples) format. Data 
      # # contains 22 channels and 500 time-points. Set the number of kernels to 1.
      # X_train = X_train.reshape(X_train.shape[0], kernels, chans, samples)
      # X_val   = X_val.reshape(X_val.shape[0], kernels, chans, samples)
   
      # print('X_train shape:', X_train.shape)
      # print(X_train.shape[0], 'train samples')
      # print(X_val.shape[0], 'val samples')

      # X_test      = x_ts_12[:,:,500:1250]
      # Y_test      = y_ts_12[:]
      # print(np.shape(X_test))
      # print(np.shape(Y_test))

      # #convert labels to one-hot encodings.
      # Y_test      = np_utils.to_categorical(Y_test-1)

      # # convert data to NCHW (trials, kernels, channels, samples) format. Data 
      # # contains 22 channels and 500 time-points. Set the number of kernels to 1.
      # X_test      = X_test.reshape(X_test.shape[0], kernels, chans, samples)

      # print('X_train shape:', X_test.shape)
      # print(X_test.shape[0], 'train samples')

      # # configure the EEGNet-8,2,16 model with kernel length of 32 samples (other 
      # # model configurations may do better, but this is a good starting point)
      # model = EEGNet(nb_classes = 2, Chans = 22, Samples = 750,
      #                dropoutRate = 0.5, kernLength = 25, F1 = 8, 
      #                D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout')
      
      # # compile the model and set the optimizers
      # model.compile(loss='categorical_crossentropy', optimizer='adam', 
      #               metrics = ['accuracy'])

      # # count number of parameters in the model
      # numParams    = model.count_params() 

      # # set a valid path for your system to record model checkpoints
      # checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1,
      #                                save_best_only=True)
  
      # # the syntax is {class_1:weight_1, class_2:weight_2,...}. Here just setting
      # # the weights all to be 1
      # class_weights = {0:1, 1:1, 2:1, 3:1}

      # history = model.fit(X_train, Y_train, batch_size = 16, epochs = 100, 
      #                 verbose = 2, validation_data=(X_val, Y_val),
      #                 callbacks=[checkpointer], class_weight = class_weights)
  
      # # Plot training & validation accuracy values
      # plt.plot(history.history['acc'])
      # plt.plot(history.history['val_acc'])
      # plt.title('Model accuracy')
      # plt.ylabel('Accuracy')
      # plt.xlabel('Epoch')
      # plt.legend(['Train', 'Test'], loc='upper left')
      # plt.show()
      # figName = 'Accuracy_A0' + str(x) + '.png'  
      # plt.savefig(figName)

      # print('\n# Evaluate on test data')
      # results = model.evaluate(X_test, Y_test, batch_size=1)
      # print('test loss, test acc:', results)

      # # acc_all[x - 1, 0] = results[0]
      # # acc_all[x - 1, 1] = results[1]

      # from keras import backend as K 
      # # Do some code, e.g. train and save model
      # K.clear_session()









EEG_Deep/Data2A/Data_A01T.mat
Filtering of Training Data Finished
EEG_Deep/Data2A/Data_A01T.mat
Filtering of Testing Data Finished
(1, 2)
Finding labels in training data
Finding labels in testing data
(120, 22, 750)
(120, 1)
(24, 22, 750)
(24, 1)
[[1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]
(1, 3)
Finding labels in training data
Finding labels in testing data
(120, 22, 750)
(120, 1)
(24, 22, 750)
(24, 1)
[[1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
(1, 4)
Finding labels in training data
Finding labels in testing data
(120, 22, 750)
(120, 1)
(24, 22, 750)
(24, 1)
[[1. 0. 0. 0.]
